In [2]:
!nvidia-smi

Wed Sep 25 08:23:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [4]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325 (from 1)
Receiving objects: 100% (325/325), 2.25 MiB | 20.72 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/kaggle/working/yolov9


In [5]:
!pip install -q roboflow

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!ls -la {HOME}/weights

In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KBFzTA5bzFUmXKAVJwox")
project = rf.workspace("final-dataset-cqduv").project("final-training-dataset")
version = project.version(2)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Final-Training-dataset-2 in yolov9:: 100%|██████████| 33370/33370 [00:06<00:00, 5048.64it/s]


In [10]:
!python detect.py --weights {HOME}/weights/gelan-c.pt --conf 0.1 --source {HOME}/yolov9/Final-Training-dataset-2/valid/images/000000386032_jpg.rf.1676fee06f7225548ff24d55437f2b81.jpg --device 0,1

detect: weights=['/kaggle/working/weights/gelan-c.pt'], source=/kaggle/working/yolov9/Final-Training-dataset-2/valid/images/000000386032_jpg.rf.1676fee06f7225548ff24d55437f2b81.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=0,1, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
Traceback (most recent call last):
  File "/kaggle/working/yolov9/detect.py", line 232, in <module>
    main(opt)
  File "/kaggle/working/yolov9/detect.py", line 227, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/kaggle/working/yolov9/detect.py", line 67, in run
    device = select_device

In [9]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/detect/exp/000000386032_jpg.rf.1676fee06f7225548ff24d55437f2b81.jpg", width=600)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/yolov9/runs/detect/exp/000000386032_jpg.rf.1676fee06f7225548ff24d55437f2b81.jpg'

In [ ]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 150 --img 640 --device 0,1 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

In [ ]:
!python train.py --resume /kaggle/input/last/other/default/1/last.pt

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/results.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/confusion_matrix.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/val_batch0_pred.jpg", width=1000)

In [ ]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0,1 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

In [ ]:
!python detect.py \
--img 1280 --conf 0.1 --device 0,1 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {dataset.location}/test/images

In [ ]:
import glob

from IPython.display import Image, display
for image_path in glob.glob(f'{HOME}/yolov9/runs/detect/exp/*.jpg'):
      display(Image(filename=image_path, width=600))

In [ ]:
import zipfile
import os

# Nama file zip yang ingin dibuat
zip_filename = '/kaggle/working/output_files.zip'

# Buat file zip
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Iterasi melalui file di direktori kerja
    for root, dirs, files in os.walk('/kaggle/working/'):
        for file in files:
            # Menambahkan file ke zip dengan path relatif
            zipf.write(os.path.join(root, file), arcname=file)

print(f'File ZIP {zip_filename} berhasil dibuat!')

In [ ]:
import os

# Cek apakah file ada di direktori
file_path = '/kaggle/working/output_files.zip'
os.path.exists(file_path)